In [21]:
import pandas as pd

In [22]:
data = pd.read_csv('/Users/yanaprynik/Work_doc/RFM2018.csv',low_memory = False)
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2018-10-01,1716.00
1,C0011810010017,13233933,2018-10-01,1489.74
2,C0011810010020,99057968,2018-10-01,151.47
3,C0011810010021,80007276,2018-10-01,146.72
4,C0011810010024,13164076,2018-10-01,104.00
...,...,...,...,...
332725,S0081810310459,14092500,2018-10-31,3801.87
332726,S0081810310461,99065678,2018-10-31,5769.88
332727,S0081810310462,19029918,2018-10-31,736.88
332728,S0081810310463,13020033,2018-10-31,1475.20


In [23]:
import datetime as dt
NOW = dt.datetime(2018,10,31)

In [26]:
rfmTable = data.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [27]:
rfmTable['frequency'].max()

204

In [28]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
rfmSegmentation = rfmTable
rfmSegmentation

,recency,frequency,monetary_value
CustomerCode,,,
02213019,7,2,3770.76
02213042,23,3,9685.48
02213071,8,2,833.00
02213081,10,1,276.98
02213082,11,2,1215.96
...,...,...,...
99099927,11,1,961.10
99099936,1,1,1521.78
99099959,9,2,1444.56


In [29]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [30]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [31]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [34]:
rfmSegmentation['RFMClass'].value_counts().sort_values()

414        2
314       33
114       60
214       60
424       63
       ...  
211     5853
344     6578
443     6726
111     9712
444    10603
Name: RFMClass, Length: 64, dtype: int64